### transaction table 
1. trasaction_1, transaction_2 결합
2. transaction_detail_1, transacrion_detail_2 결합
3. 1번에서 나온 결과 2번에서 나온 결과를 조인 결합 (left)

In [1]:
import pandas as pd
import mod_sql

In [2]:
_db = mod_sql.Database()

In [5]:
sql = """
SELECT * FROM transaction_1 
left join transaction_detail_1 
ON transaction_1.transaction_id = transaction_detail_1.transaction_id
"""
_db.executeAll(sql)

,transaction_id,price,payment_date,customer_id,detail_id,transaction_detail_1.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7140,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1
7141,T0000006895,85000,2019-07-31 21:52:48,AS339451,7140,T0000006895,S002,1
7142,T0000006897,85000,2019-07-31 23:39:35,TS624738,7142,T0000006897,S002,1


In [ ]:
# transaction_1, transaction_2 결합 (유니언)
sql = """
SELECT * FROM transaction_1 
UNION 
SELECT * FROM transaction_2
"""
_db.executeAll(sql)

In [ ]:
sql = """
SELECT * FROM transaction_detail_1 
UNION 
SELECT * FROM transaction_detail_2
"""
_db.executeAll(sql)

In [8]:
# 3번 과정 left join 결합
sql = """
select * from 
(SELECT * FROM transaction_1 
UNION 
SELECT * FROM transaction_2) as transaction
left join 
(SELECT * FROM transaction_detail_1 
UNION 
SELECT * FROM transaction_detail_2) as transaction_detail
ON transaction.transaction_id = transaction_detail.transaction_id
"""
_db.executeAll(sql)

,transaction_id,price,payment_date,customer_id,detail_id,transaction_detail.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1


In [17]:
sql = """
select * from transaction_detail_1
"""
_db.executeAll(sql)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


In [26]:
## left join transaction_detail_1, item_master 두개의 테이블을 조인 결합
## join 결합을 한 데이터에서 quantity * item_price 데이터 컬럼(price)을 추가 
## 결과적으로 출력 컬럼은 
# transaction_id(transaction_detail_1에 존재), 
# quantity(transaction_detail_1에 존재), 
# item_price(item_master에 존재), 
# price(새로 생성된 파생변수) 을 출력
sql = """
SELECT 
    `transaction_id`, `quantity`, `item_price`, tr_d.`item_id`, 
    (`quantity` * `item_price`) as price 
FROM transaction_detail_1 as tr_d
    left join item_master as i_m
ON tr_d.`item_id` = i_m.`item_id` 
"""
_db.executeAll(sql)

,transaction_id,quantity,item_price,item_id,price
0,T0000000113,1,210000,S005,210000
1,T0000000114,1,50000,S001,50000
2,T0000000115,1,120000,S003,120000
3,T0000000116,1,210000,S005,210000
4,T0000000117,2,85000,S002,170000
...,...,...,...,...,...
7139,T0000006894,1,180000,S004,180000
7140,T0000006895,1,85000,S002,85000
7141,T0000006896,2,50000,S001,100000
7142,T0000006897,1,85000,S002,85000


1. transaction_1에 있는 payment_date 컬럼이 datetime 형태
2. 파생변수(payment_month) payment_date 값에서 년과 월만 따로 추출해서 삽입
3. 월별로 그룹화 price 평균 , 합계 값을 출력

In [6]:
sql = """
SELECT 
payment_date, 
DATE_FORMAT(`payment_date`, '%%Y%%m') as payment_month, 
AVG(`price`) as AVG, 
SUM(`price`) as SUM 
from transaction_1 
GROUP BY payment_month
"""
_db.executeAll(sql)

,payment_date,payment_month,AVG,SUM
0,2019-02-01 01:36:57,201902,144701.8970,160185000
1,2019-03-01 00:20:55,201903,142424.5115,160370000
2,2019-04-01 00:00:06,201904,142802.4911,160510000
3,2019-05-01 01:46:23,201905,139515.2603,155420000
4,2019-06-01 00:38:49,201906,144392.6056,164030000
5,2019-07-01 00:19:44,201907,144715.8609,170620000


In [30]:
sql = """
select * from transaction_1
"""
df = _db.executeAll(sql)


### transaction_1의 데이터를 select을 이용하여 데이터를 받아와서 df 라는 변수 삽입
1. df(데이터프레임)을 이용하여 월별 price의 평균과 합계를 구하시오.(sql 에서 했던 작업)
2. pandas를 이용하여 출력을 해보면 될거 같습니다. 
3. payment_month 파생변수 생성. 
4. 그룹화 price 평균과 합계를 확인.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6786 entries, 0 to 6785
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  6786 non-null   object        
 1   price           6786 non-null   int64         
 2   payment_date    6786 non-null   datetime64[ns]
 3   customer_id     6786 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 212.2+ KB


In [ ]:
df["payment_month"] = df["payment_date"].dt.strftime("%Y%m")    ## 년도와 월의 데이터만 추출해서 컬럼에 삽입
df.head()

In [ ]:
df.groupby('payment_month').sum()['price']

In [ ]:
df.groupby('payment_month').mean()['price']

In [37]:
df.groupby('payment_month').agg(["mean", "sum"])['price']

,mean,sum
payment_month,,
201902,144701.897019,160185000
201903,142424.511545,160370000
201904,142802.491103,160510000
201905,139515.260323,155420000
201906,144392.605634,164030000
201907,144715.860899,170620000


In [38]:
sql = "select * from transaction_detail_1"
sql2 = "select * from item_master"

df_2 = _db.executeAll(sql)
df_3 = _db.executeAll(sql2)

In [ ]:
## df_2 -> transaction_detail_1 정보
## df_3 -> item_master 정보
## 두 데이터프레임을 join  결합
total_df = pd.merge(df_2, df_3, on="item_id", how="left")
total_df.head(1)

In [43]:
total_df["price"] = total_df["quantity"] * total_df["item_price"]
total_df

,detail_id,transaction_id,item_id,quantity,item_name,item_price,price
0,0,T0000000113,S005,1,PC-E,210000,210000
1,1,T0000000114,S001,1,PC-A,50000,50000
2,2,T0000000115,S003,1,PC-C,120000,120000
3,3,T0000000116,S005,1,PC-E,210000,210000
4,4,T0000000117,S002,2,PC-B,85000,170000
...,...,...,...,...,...,...,...
7139,7139,T0000006894,S004,1,PC-D,180000,180000
7140,7140,T0000006895,S002,1,PC-B,85000,85000
7141,7141,T0000006896,S001,2,PC-A,50000,100000
7142,7142,T0000006897,S002,1,PC-B,85000,85000
